# 5.1 tf.keras.layers.Conv2D による CNN の実装

<code>
tf.keras.layers.Conv2D(
    filters,
    kernel-size,
    stride=(1,1),
    padding='valid',
    data_format=None,
    dilation_rate=(1,1),
    groups=1,
    activation=None,
    use_bias=True,
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None
    )
</code>

<code>
tf.keras.layers.MaxPool2D(
    pool_size=(2,2),
    strides=None,
    padding='valid',
    data_format=None
    )
</code>

<code>
tf.keras.layers.Dropout(
    rate,
    noise_shape=None,
    seed=None
    )
</code>

In [1]:
# 5-01-1 Fashion-MNIST データの読み込みと前処理
# p.252
# [自分へのメモ]前処理したデータは、変数に上書きせずに、別の変数に保存する

import tensorflow as tf
from tensorflow  import keras

(x_train, t_train), (x_test, t_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train_norm = x_train.astype('float32') / 255.
x_test_norm = x_test.astype('float32') / 255.

x_train_rs = x_train_norm.reshape(-1, 28, 28, 1)
x_test_rs = x_test_norm.reshape(-1, 28, 28, 1)

4431872/4422102 [==============================] - 0s 0us/step


In [2]:
# 5-01-2 CNN モデルを生成するクラス
# p.259

class CNN(tf.keras.Model):
    
    def __init__(self, output_dim):
        super().__init__()
        weight_decay = 1e-4
        self.c1 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=(3,3),
            padding='same',
            input_shape=x_train[0].shape,
            kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
            activation='relu'
        )
        self.c2 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=(3,3),
            padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
            activation='relu'
        )
        self.p1 = tf.keras.layers.MaxPooling2D(
            pool_size=(2,2)
        )
        self.c3 = tf.keras.layers.Conv2D(
            filters=16,
            kernel_size=(3,3),
            padding='same',
            kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
            activation='relu'
        )
        self.p2 = tf.keras.layers.MaxPooling2D(
            pool_size=(2,2)
        )
        self.d1 = tf.keras.layers.Dropout(0.4)
        self.f1 = tf.keras.layers.Flatten()
        self.l1 = tf.keras.layers.Dense(
            128,
            activation='relu'
        )
        self.l2 = tf.keras.layers.Dense(
            output_dim,
            activation='softmax'
        )
        self.ls = [ self.c1, self.c2, self.p1, self.c3, self.p2, self.d1, self.f1, self.l1, self.l2 ]
        
    def call(self, x):
        for layer in self.ls:
            x = layer(x)
        return x

In [3]:
# 5-01-3 損失関数の定義
# p.263

cce = keras.losses.SparseCategoricalCrossentropy()

def loss(t, y):
    return cce(t,y)

In [4]:
# 5-01-4 Stochastic Gradient Descend を使用するオプティマイザー
# p.263

optimizer = keras.optimizers.SGD(learning_rate=0.1)

# 損失を記録するオブジェクト
train_loss = keras.metrics.Mean()

# 精度を記録するオブジェクト
train_acc = keras.metrics.SparseCategoricalAccuracy()

# validationの損失を記録するオブジェクト
val_loss = keras.metrics.Mean()

# validationの精度を記録する
val_acc = keras.metrics.SparseCategoricalAccuracy()

In [5]:
import tensorflow as tf
# p.264
def train_step(x, t):
    '''
    学習を1回行う
    
    Parameters:
      x (ndarray(float32)): 訓練データ
      t (ndarray(float32)): 正解ラベル
      
    Returns:
      ステップごとのクロスエントロピー誤差
      
      '''
    
    with tf.GradientTape() as tape:
        outputs = model(x)
        tmp_loss = loss(t, outputs)
    grads = tape.gradient(tmp_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # 損失と精度を記録
    train_loss(tmp_loss)
    train_acc(t, outputs)

In [6]:
# 5-01-5 検証データによる評価を行う
# p.265
def val_step(x, t):
    preds = model(x)
    tmp_loss = loss(t, preds)
    val_loss(tmp_loss)
    val_acc(t, preds)

### 学習の実行

In [7]:
# 5-01-6 訓練データを検証データを用意する
# p.266
from sklearn.model_selection import train_test_split

x_tr, x_val, t_tr, t_val = train_test_split(x_train_rs, t_train, test_size=0.2)

In [8]:
print(x_tr.shape)
print(x_val.shape)
print(t_tr.shape)
print(t_val.shape)

(48000, 28, 28, 1)
(12000, 28, 28, 1)
(48000,)
(12000,)


In [9]:
# 5-01-7 モデルを生成して学習する
# p.266
from sklearn.utils import shuffle

epochs = 100
batch_size = 64
steps = x_tr.shape[0] // batch_size
steps_val = x_val.shape[0] // batch_size

model = CNN(10)

In [10]:
# 学習を行う
# p.267
# [自分へのメモ] バグだと思われる部分を変更している。

for epoch in range(epochs):
    x_, t_ = shuffle(x_tr, t_tr, random_state=1)
    
    for step in range(steps):
        start = step * batch_size
        end = start + batch_size
        train_step(x_[start:end], t_[start:end])
        
    for step_val in range(steps_val):
        start = step_val * batch_size
        end = start + batch_size
        #val_step(x_[start:end], t_[start:end])   ### !!! Bug
        val_step(x_val[start:end], t_val[start:end])
        
    print(f'epoch: {(epoch+1)} '
          f'train_loss: {train_loss.result():.4f} '
          f'train_acc:{train_acc.result():.4f} '
          f'val_loss: {val_loss.result():.4f} '
          f'val_acc: {val_acc.result():.4f}')

epoch: 1 train_loss: 0.6244 train_acc:0.7717 val_loss: 0.5004 val_acc: 0.8079
epoch: 2 train_loss: 0.4996 train_acc:0.8172 val_loss: 0.4429 val_acc: 0.8334
epoch: 3 train_loss: 0.4387 train_acc:0.8389 val_loss: 0.4063 val_acc: 0.8489
epoch: 4 train_loss: 0.3993 train_acc:0.8531 val_loss: 0.3811 val_acc: 0.8591
epoch: 5 train_loss: 0.3705 train_acc:0.8636 val_loss: 0.3626 val_acc: 0.8662
epoch: 6 train_loss: 0.3476 train_acc:0.8719 val_loss: 0.3493 val_acc: 0.8716
epoch: 7 train_loss: 0.3286 train_acc:0.8788 val_loss: 0.3395 val_acc: 0.8756
epoch: 8 train_loss: 0.3125 train_acc:0.8847 val_loss: 0.3317 val_acc: 0.8789
epoch: 9 train_loss: 0.2982 train_acc:0.8900 val_loss: 0.3260 val_acc: 0.8813
epoch: 10 train_loss: 0.2855 train_acc:0.8946 val_loss: 0.3208 val_acc: 0.8838
epoch: 11 train_loss: 0.2739 train_acc:0.8989 val_loss: 0.3170 val_acc: 0.8858
epoch: 12 train_loss: 0.2632 train_acc:0.9029 val_loss: 0.3145 val_acc: 0.8873
epoch: 13 train_loss: 0.2532 train_acc:0.9068 val_loss: 0.313

### テストデータによるモデルの評価

In [11]:
# 5-01-8 テストデータによるモデルの評価
# p.269

test_loss = keras.metrics.Mean()
test_acc = keras.metrics.SparseCategoricalAccuracy()

def test_step(x, t):
    '''
    Parameters:
      x (ndarray(float32)): test data
      t (ntarray(float32)): label
    '''
    preds = model(x)
    tmp_loss = loss(t, preds)
    test_loss(tmp_loss)
    test_acc(t, preds)

In [12]:
test_step(x_test_rs, t_test)

print(f'test_loss: {test_loss.result():.4f} '
      f'test_acc: {test_acc.result():.4f}')

test_loss: 1.0943 test_acc: 0.9048
